## First, imports:

In [ ]:
%load_ext autoreload
%autoreload 2

%config IPCompleter.greedy=True

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy import units
import importlib

In [ ]:
import os
import sys

apt_path = os.path.abspath(os.path.join('..', 'apostletools'))
sys.path.append(apt_path)
import dataset_comp
import snapshot

In [ ]:
importlib.reload(dataset_comp)
importlib.reload(snapshot)

# Definitions of Satellite Galaxies and Isolated Galaxies

## Motivation

To illustrate the two definitions, I plot the subhalos, projected onto a plane, and color satellites and isolated galaxies with different colors. 

---

## Set Parameters for the Plots

Choose the snapshot and the simulation, and define M31 and MW in each simulation. Also, set the colors used for each simulation:

In [ ]:
sim_id = "V1_MR_curvaton_p082_fix"
snap_id = 127

m31 = (1,0)
mw = (1,1)

Set the maximum distances for satellite galaxies and isolated galaxies (for the definitions by distance):

In [ ]:
maxds = 300
maxdi = 2000 # Maximum distance from LG centre for isolated

---

## Retrieve Data

Get centres of potentials of subhalos and compute them wrt. the LG centre:

In [ ]:
# Get data:
snap = snapshot.Snapshot(sim_id, snap_id)
cops = snap.get_subhalos("CentreOfPotential")
LG_centre = dataset_comp.compute_LG_centre(snap, m31, mw)

# Centre on LG:
cops = dataset_comp.periodic_wrap(snap, LG_centre, cops) - LG_centre

# Convert units:
cops = cops * units.cm.to(units.Mpc)

### Compute Projections

Find two orthogonal planes that intersect both M31 and MW. Denote the two vectors normal to these planes by n1 and n2. The two (arbitrary) parameters a and b determine the orientation angle of the plane normal to n1. 

In [ ]:
cop_m31 = cops[snap.index_of_halo(m31[0], m31[1])]
cop_mw = cops[snap.index_of_halo(mw[0], mw[1])]
# Compute the unit separation vector:
r = cop_m31 - cop_mw
r = r / np.linalg.norm(r)

# Compute a unit vector orthogonal to r, n1:
a = 1; b = 1
n1 = np.array([-1/r[0] * (a*r[1] + b*r[2]), a, b])
n1 = n1 / np.linalg.norm(n1)

# Compute another unit vector orthogonal to both r and n1:
n2 = np.cross(r, n1)

In [ ]:
# Sanity checks:
print(cop_m31)
print(cop_mw)
print(np.linalg.norm(r))
print(np.linalg.norm(n2))
print(np.dot(n1, r))
print(np.dot(n2, r))
print(np.dot(n1, n2))

Project the subhalos onto the two planes:

In [ ]:
# Project onto the plane orthogonal to n1:
cops_n1 = np.column_stack((np.dot(cops, r), np.dot(cops, n2)))
m31_n1 = np.array((np.dot(cop_m31, r), np.dot(cop_m31, n2)))
mw_n1 = np.array((np.dot(cop_mw, r), np.dot(cop_mw, n2)))

# Project onto the plane orthogonal to n2:
cops_n2 = np.column_stack((np.dot(cops, r), np.dot(cops, n1)))
m31_n2 = np.array((np.dot(cop_m31, r), np.dot(cop_m31, n1)))
mw_n2 = np.array((np.dot(cop_mw, r), np.dot(cop_mw, n1)))

### Create a Dictionary for Selections

Compute the masking arrays to select satellites or isolated galaxies from subhalos, and save these to the dictionary "selections":

In [ ]:
selections = {}

# To prune potential spurious:
mask_prune_vmax = dataset_comp.prune_vmax(snap)

# Split into satellites and isolated:
masks_sat_d, mask_isol_d = dataset_comp.split_satellites_by_distance(
    snap, m31, mw, max_dist_sat=maxds, max_dist_isol=maxdi)
masks_sat_gn, mask_isol_gn = dataset_comp.split_satellites_by_group_number(
    snap, m31, mw)

selections['By_distance'] = {'M31_satellites': masks_sat_d[0][mask_prune_vmax],
                             'MW_satellites': masks_sat_d[1][mask_prune_vmax],
                             'Isolated': mask_isol_d[mask_prune_vmax]}
selections['By_GN'] = {'M31_satellites': masks_sat_gn[0][mask_prune_vmax],
                       'MW_satellites': masks_sat_gn[1][mask_prune_vmax],
                       'Isolated': mask_isol_gn[mask_prune_vmax]}

---

## Plot Subhalo Positions

Plot two columns for satellite/isolated galaxy selections by group number and by distance. Plot two rows for the two orthogonal plane projections.

First, set figure parameters:

In [ ]:
# Choose font sizes:
parameters = {'axes.titlesize': 10,
              'axes.labelsize': 9,
              'xtick.labelsize': 6,
              'ytick.labelsize': 6,
              'legend.fontsize': 8}

# Side length (in Mpc) of each subfigure:
ax_size = 5

s = 0.1 # marker size
lw = 0.3 # line width

In [ ]:
# Set fonts:
plt.rcParams.update(parameters)
plt.tight_layout()

fig, axes = plt.subplots(nrows=2, ncols=2, sharex='all', sharey='row',
                         subplot_kw=dict(box_aspect=1), figsize=(4.9,5))
plt.subplots_adjust(wspace=0.04)
plt.subplots_adjust(hspace=0.04)

# Set axis labels:
axes[1,0].set_xlabel("$x [\mathrm{Mpc}]$")
axes[1,1].set_xlabel("$x [\mathrm{Mpc}]$")
axes[0,0].set_ylabel("$y [\mathrm{Mpc}]$")
axes[1,0].set_ylabel("$z [\mathrm{Mpc}]$")

# Set axis limits:
axes[0,0].set_xlim(-ax_size/2, ax_size/2)
axes[0,0].set_ylim(-ax_size/2, ax_size/2)
axes[1,1].set_xlim(-ax_size/2, ax_size/2)
axes[1,1].set_ylim(-ax_size/2, ax_size/2)

axes[0,0].set_title("By group number")
axes[0,1].set_title("By distance")

# Plot colored by group number:
axes[0,0].scatter(cops_n1[selections['By_GN']['Isolated']][:,0],
                  cops_n1[selections['By_GN']['Isolated']][:,1],
                  c='gray', s=s)
axes[0,0].scatter(cops_n1[selections['By_GN']['M31_satellites']][:,0],
                  cops_n1[selections['By_GN']['M31_satellites']][:,1],
                  c='pink', s=s)
axes[0,0].scatter(cops_n1[selections['By_GN']['MW_satellites']][:,0],
                  cops_n1[selections['By_GN']['MW_satellites']][:,1],
                  c='lightblue', s=s)

axes[1,0].scatter(cops_n2[selections['By_GN']['Isolated']][:,0],
                  cops_n2[selections['By_GN']['Isolated']][:,1],
                  c='gray', s=s)
axes[1,0].scatter(cops_n2[selections['By_GN']['M31_satellites']][:,0],
                  cops_n2[selections['By_GN']['M31_satellites']][:,1],
                  c='pink', s=s)
axes[1,0].scatter(cops_n2[selections['By_GN']['MW_satellites']][:,0],
                  cops_n2[selections['By_GN']['MW_satellites']][:,1],
                  c='lightblue', s=s)

# Plot colored by distance:
axes[0,1].scatter(cops_n1[selections['By_distance']['Isolated']][:,0],
                  cops_n1[selections['By_distance']['Isolated']][:,1],
                  c='gray', s=s)
axes[0,1].scatter(cops_n1[selections['By_distance']['M31_satellites']][:,0],
                  cops_n1[selections['By_distance']['M31_satellites']][:,1],
                  c='pink', s=s)
axes[0,1].scatter(cops_n1[selections['By_distance']['MW_satellites']][:,0],
                  cops_n1[selections['By_distance']['MW_satellites']][:,1],
                  c='lightblue', s=s)

axes[1,1].scatter(cops_n2[selections['By_distance']['Isolated']][:,0],
                  cops_n2[selections['By_distance']['Isolated']][:,1],
                  c='gray', s=s)
axes[1,1].scatter(cops_n2[selections['By_distance']['M31_satellites']][:,0],
                  cops_n2[selections['By_distance']['M31_satellites']][:,1],
                  c='pink', s=s)
axes[1,1].scatter(cops_n2[selections['By_distance']['MW_satellites']][:,0],
                  cops_n2[selections['By_distance']['MW_satellites']][:,1],
                  c='lightblue', s=s)

### Add annotations

Draw circles of 300 kpc around the centrals and a circle of 2 Mpc around the LG.

In [ ]:
def circle(centre_x, centre_y, r, n):
    t = 4*np.pi/n * np.arange(n/2)
    x = centre_x + r * np.cos(t)
    y = centre_y + r * np.sin(t)
    return x, y

In [ ]:
# Add annotations:
for m31, mw, axx, axy in zip([m31_n1, m31_n1, m31_n2, m31_n2],
                             [mw_n1, mw_n1, mw_n2, mw_n2],
                             [0,0,1,1], [0,1,0,1]):
    # Plot M31 and circle:
    axes[axx, axy].scatter(m31[0], m31[1], c='red', s=s)
    x_circ, y_circ = circle(m31[0], m31[1], 0.3, 10000)    
    axes[axx, axy].plot(x_circ, y_circ, c='red', linestyle='dashed', linewidth=lw)
    
    # Plot MW and circle:
    axes[axx, axy].scatter(mw[0], mw[1], c='blue', s=s)
    x_circ, y_circ = circle(mw[0], mw[1], 0.3, 10000)
    axes[axx, axy].plot(x_circ, y_circ, c='blue', linestyle='dashed', linewidth=lw)
    
    # Plot 2 Mpc circle around LG centre:
    x_circ, y_circ = circle(0, 0, 2, 10000)
    axes[axx, axy].plot(x_circ, y_circ, c='k', linestyle='dashed', linewidth=lw)
    
fig

### Save the Figure

In [ ]:
# Construct saving location:
filename = 'Distinction_comparison_{}.png'.format(sim_id)    
home = os.path.abspath(os.path.join('..'))
path = os.path.join(home,'Figures', 'MediumResolution')
filename = os.path.join(path, filename)
    
fig.savefig(filename, dpi=300)